In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import numpy as np
import keras
from keras.models import load_model
import time

from Source import RM
import tensorflow as tf

Using TensorFlow backend.


In [3]:
m = 4
r = 1
length = 2**m

G = RM.G_matrix(length, m, r)
G_p = RM.G_prime(G, m)

if r == 1:
    masking_length=0
else:
    masking_length=int(np.math.factorial(m)/(np.math.factorial(2)*np.math.factorial(m-2)))

In [4]:
def add_masking(received_code, msg_size):
    masking = np.ones((msg_size,2))
    if np.var(received_code) > 1.9:
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.45:
        masking[:,1] = -1*masking[:,1]
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.225:
        masking[:,0] = -1*masking[:,0]
        received_code = np.hstack((masking, received_code))
    else:
        masking = -1 * masking
        received_code = np.hstack((masking, received_code))
    return received_code

In [5]:
with tf.device('/cpu:0'):
    model = load_model('Model/model_(1,4)_p6_softmax.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
ad_time = 0
rm_time = 0

In [7]:
msg = np.random.randint(2, size = (1,m+r))
send_code, recived_code = RM.Encoding(msg, G, 3, length)

In [8]:
with tf.device('/cpu:0'):
    start =  time.time()
    for i in range(10**4):
        a = model.predict(recived_code)
    end =  time.time()    
    print(end-start)

6.402878046035767


In [9]:

start =  time.time()
for i in range(10**4):
    rm_estimated_code = RM.Decoding(recived_code, m, length, r, G, G_p)
end = time.time()
print(end-start)

C:\Users\MCC\Documents\jupyter\rm_code\Source\RM.py:46: RuntimeWarning: overflow encountered in add
  sum_code = recived_code[:,k] + recived_code[:,k+1*(2**j)]
C:\Users\MCC\Documents\jupyter\rm_code\Source\RM.py:46: RuntimeWarning: invalid value encountered in add
  sum_code = recived_code[:,k] + recived_code[:,k+1*(2**j)]
C:\Users\MCC\Documents\jupyter\rm_code\Source\RM.py:47: RuntimeWarning: invalid value encountered in subtract
  subtract_code = recived_code[:,k] - recived_code[:,k+1*(2**j)]
C:\Users\MCC\Documents\jupyter\rm_code\Source\RM.py:56: RuntimeWarning: invalid value encountered in greater
  value = np.where(value > 0, 0, 1)


9.901478052139282


In [1]:
import timeit
def a():
    setup_code = '''
import numpy as np 
import keras
from keras.models import load_model
from Source import RM

m = 4
r = 1
length = 2**m

G = RM.G_matrix(length, m, r)
G_p = RM.G_prime(G, m)


msg = np.random.randint(2, size = (1,m+r))
send_code, recived_code = RM.Encoding(msg, G, 3, length)'''
    test_code ='RM.Decoding(recived_code, m, length, r, G, G_p)'
    
    times = timeit.timeit(setup=setup_code, stmt=test_code, number = 100000)
    
    print(times)

In [3]:
def b():
    setup_code = '''
import numpy as np 
import keras
from keras.models import load_model
from Source import RM

m = 4
r = 1
length = 2**m

G = RM.G_matrix(length, m, r)
G_p = RM.G_prime(G, m)


msg = np.random.randint(2, size = (1,m+r))
send_code, recived_code = RM.Encoding(msg, G, 3, length)
model = load_model('Model/model_(1,4)_p6_softmax.h5')'''
    test_code='c = model.predict(recived_code)'
    
    times = timeit.timeit(setup=setup_code, stmt=test_code, number = 1000)
    
    print(times)

In [8]:
a()

1.001138999999995


In [9]:
b()

3.1268862999999953
